In [1]:
import sys
sys.path.insert(0,"../../Code/Classes")
from DataClass import DataLoader

import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
def split_image(img,n_splits):
    #Assuming squares
    sz_xy = img.shape[0]
    sz_sm = int(sz_xy/n_splits)
    lay = img.shape[2]
    out = np.zeros((n_splits**2,sz_sm,sz_sm,lay))
    for i in range(n_splits):
        for j in range(n_splits):
            c_x = sz_sm*i
            c_y = sz_sm*j
            out[n_splits*i+j,:,:,:] = img[c_x:c_x+sz_sm,c_y:c_y+sz_sm,:]
    return out

def unsplit_image(imgs):
    num_splits = imgs.shape[0]
    s_splits = int(np.sqrt(num_splits))
    sm_res = imgs.shape[1]
    channels = imgs.shape[3]
    
    big_res = sm_res*s_splits
    
    out = np.zeros((big_res,big_res,channels))
    for i in range(s_splits):
        for j in range(s_splits):
            c_x = sm_res*i
            c_y = sm_res*j
            out[c_x:c_x+sm_res,c_y:c_y+sm_res,:]=imgs[s_splits*i+j,:,:,:]
    return out            

In [3]:
dataloader = DataLoader()
dataloader.load_std_folder("../../../Data/STD_Folder")
dataloader.parse_biclass()

In [4]:
size_f = 1024
size_r = 16
#Assuming square
splits = int(size_f/size_r)

In [5]:
full_num_img = len(dataloader.classfiles)
im_indices = [a for a in range(full_num_img)]

In [6]:
from sklearn.model_selection import train_test_split

samp_per_img = 256

split_range = [a for a in range(splits*splits)]
samp = train_test_split(split_range,train_size = samp_per_img)[0]

X = np.zeros((full_num_img*samp_per_img,size_r,size_r,1))
y = np.zeros((full_num_img*samp_per_img,size_r,size_r),bool)
for a in im_indices:
    c_ind = a*samp_per_img
    tmp = dataloader.get_data("depth",[a])[0]
    tmp = np.moveaxis([tmp],(0,1,2),(2,0,1))
    c_img = split_image(tmp,splits)
    c_class_tmp = dataloader.get_data("class",[a])[0]
    c_class = split_image(cv2.resize(c_class_tmp,(size_f,size_f)),splits)[:,:,:,0].astype(bool)
    X[c_ind:c_ind+samp_per_img,:,:,:] = c_img[samp]
    y[c_ind:c_ind+samp_per_img,:,:] = c_class[samp]
y = np.moveaxis(np.asarray([y],bool),(0,1,2,3),(3,0,1,2))

X = np.asarray([(a-np.min(a,0))/(np.max(a)-np.min(a)) for a in X])

In [7]:
#Checking class distribution around 0.5
np.sum(y)/y.size

0.523796757565269

In [8]:
#Splitting into train and test data
test_size = 0.1
val_size = 0.1

shape_range = [a for a in range(X.shape[0])]

t_ind,test_ind = train_test_split(shape_range,test_size=test_size)
train_ind,val_ind = train_test_split(t_ind,test_size=val_size)

X_train,y_train = X[train_ind],y[train_ind]
X_val,y_val = X[val_ind],y[val_ind]
X_test,y_test = X[test_ind],y[test_ind]

In [9]:
#Determining max num filters
min_res = 4
reduction_goal = 0.5
max_filters = int(reduction_goal*size_r**2/(min_res**2))
max_filters

8

In [10]:
#16x16x1 -> 4x4x8

from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32,kernel_size=7,input_shape = [size_r,size_r,1],
                              activation="tanh", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=8,kernel_size=3,activation="relu", padding="same"))

#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=32,kernel_size=2,activation="relu", padding="same"))
model.add(keras.layers.Conv2D(1,kernel_size=3,activation="sigmoid", padding="same"))

In [11]:
model.compile(loss="mean_squared_error",metrics = [keras.metrics.RootMeanSquaredError()],
             optimizer="nadam")

In [12]:
history = model.fit(X_train,X_train,epochs=100,validation_data=(X_val,X_val),batch_size=32)

Epoch 1/100
512/512 [==============================] - 10s 19ms/step - loss: 0.0131 - root_mean_squared_error: 0.1146 - val_loss: 0.0046 - val_root_mean_squared_error: 0.0675
Epoch 2/100
512/512 [==============================] - 9s 18ms/step - loss: 0.0036 - root_mean_squared_error: 0.0596 - val_loss: 0.0023 - val_root_mean_squared_error: 0.0476
Epoch 3/100
512/512 [==============================] - 9s 18ms/step - loss: 0.0023 - root_mean_squared_error: 0.0475 - val_loss: 0.0017 - val_root_mean_squared_error: 0.0413
Epoch 4/100
512/512 [==============================] - 9s 18ms/step - loss: 0.0018 - root_mean_squared_error: 0.0419 - val_loss: 0.0013 - val_root_mean_squared_error: 0.0361
Epoch 5/100
512/512 [==============================] - 9s 18ms/step - loss: 0.0015 - root_mean_squared_error: 0.0386 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0442
Epoch 6/100
512/512 [==============================] - 9s 18ms/step - loss: 0.0013 - root_mean_squared_error: 0.0358 - val_loss: 

512/512 [==============================] - 10s 19ms/step - loss: 2.5831e-04 - root_mean_squared_error: 0.0161 - val_loss: 3.5307e-04 - val_root_mean_squared_error: 0.0188
Epoch 47/100
512/512 [==============================] - 9s 18ms/step - loss: 2.5472e-04 - root_mean_squared_error: 0.0160 - val_loss: 2.6355e-04 - val_root_mean_squared_error: 0.0162
Epoch 48/100
512/512 [==============================] - 9s 18ms/step - loss: 2.5273e-04 - root_mean_squared_error: 0.0159 - val_loss: 3.2225e-04 - val_root_mean_squared_error: 0.0180
Epoch 49/100
512/512 [==============================] - 9s 18ms/step - loss: 2.5774e-04 - root_mean_squared_error: 0.0161 - val_loss: 2.5797e-04 - val_root_mean_squared_error: 0.0161
Epoch 50/100
512/512 [==============================] - 9s 18ms/step - loss: 2.6333e-04 - root_mean_squared_error: 0.0162 - val_loss: 2.2711e-04 - val_root_mean_squared_error: 0.0151
Epoch 51/100
512/512 [==============================] - 10s 19ms/step - loss: 2.2630e-04 - root_m

KeyboardInterrupt: 

In [22]:
#Saving model
model.save("Depth_4x4x8_autoencoder.h5")

In [23]:
#Test error
np.sqrt(np.mean((model.predict(X_test)-X_test)**2))

0.3198180238912796

In [24]:
size_f = 1024
size_r = 32
#Assuming square
splits = int(size_f/size_r)

full_num_img = len(dataloader.classfiles)
im_indices = [a for a in range(full_num_img)]


from sklearn.model_selection import train_test_split

samp_per_img = 128

split_range = [a for a in range(splits*splits)]
samp = train_test_split(split_range,train_size = samp_per_img)[0]

X = np.zeros((full_num_img*samp_per_img,size_r,size_r,1))
y = np.zeros((full_num_img*samp_per_img,size_r,size_r),bool)
for a in im_indices:
    c_ind = a*samp_per_img
    tmp = dataloader.get_data("depth",[a])[0]
    tmp = np.moveaxis([tmp],(0,1,2),(2,0,1))
    c_img = split_image(tmp,splits)
    c_class_tmp = dataloader.get_data("class",[a])[0]
    c_class = split_image(cv2.resize(c_class_tmp,(size_f,size_f)),splits)[:,:,:,0].astype(bool)
    X[c_ind:c_ind+samp_per_img,:,:,:] = c_img[samp]
    y[c_ind:c_ind+samp_per_img,:,:] = c_class[samp]
y = np.moveaxis(np.asarray([y],bool),(0,1,2,3),(3,0,1,2))

X = np.asarray([(a-np.min(a,0))/(np.max(a)-np.min(a)) for a in X])


#Splitting into train and test data
test_size = 0.1
val_size = 0.1

shape_range = [a for a in range(X.shape[0])]

t_ind,test_ind = train_test_split(shape_range,test_size=test_size)
train_ind,val_ind = train_test_split(t_ind,test_size=val_size)

X_train,y_train = X[train_ind],y[train_ind]
X_val,y_val = X[val_ind],y[val_ind]
X_test,y_test = X[test_ind],y[test_ind]

In [25]:
#32x32x1 -> 8x8x8

from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32,kernel_size=7,input_shape = [size_r,size_r,1],
                              activation="tanh", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=8,kernel_size=3,activation="relu", padding="same"))

#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(1,kernel_size=3,activation="sigmoid", padding="same"))

In [26]:
model.compile(loss="mean_squared_error",metrics = [keras.metrics.RootMeanSquaredError()],
             optimizer="nadam")

In [27]:
history = model.fit(X_train,X_train,epochs=100,validation_data=(X_val,X_val),batch_size=32)

Epoch 1/100
256/256 [==============================] - 15s 57ms/step - loss: 0.0162 - root_mean_squared_error: 0.1275 - val_loss: 0.0032 - val_root_mean_squared_error: 0.0563
Epoch 2/100
256/256 [==============================] - 14s 56ms/step - loss: 0.0033 - root_mean_squared_error: 0.0576 - val_loss: 0.0025 - val_root_mean_squared_error: 0.0500
Epoch 3/100
256/256 [==============================] - 14s 56ms/step - loss: 0.0024 - root_mean_squared_error: 0.0488 - val_loss: 0.0020 - val_root_mean_squared_error: 0.0442
Epoch 4/100
256/256 [==============================] - 14s 57ms/step - loss: 0.0020 - root_mean_squared_error: 0.0444 - val_loss: 0.0014 - val_root_mean_squared_error: 0.0376
Epoch 5/100
256/256 [==============================] - 15s 57ms/step - loss: 0.0016 - root_mean_squared_error: 0.0395 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0330
Epoch 6/100
256/256 [==============================] - 14s 56ms/step - loss: 0.0013 - root_mean_squared_error: 0.0358 - val_l

Epoch 46/100
256/256 [==============================] - 15s 57ms/step - loss: 2.7498e-04 - root_mean_squared_error: 0.0166 - val_loss: 7.0323e-04 - val_root_mean_squared_error: 0.0265
Epoch 47/100
256/256 [==============================] - 14s 57ms/step - loss: 2.7804e-04 - root_mean_squared_error: 0.0167 - val_loss: 2.6808e-04 - val_root_mean_squared_error: 0.0164
Epoch 48/100
256/256 [==============================] - 15s 57ms/step - loss: 2.6802e-04 - root_mean_squared_error: 0.0164 - val_loss: 4.0169e-04 - val_root_mean_squared_error: 0.0200
Epoch 49/100
256/256 [==============================] - 15s 57ms/step - loss: 2.6987e-04 - root_mean_squared_error: 0.0164 - val_loss: 2.6359e-04 - val_root_mean_squared_error: 0.0162
Epoch 50/100
256/256 [==============================] - 15s 58ms/step - loss: 2.7427e-04 - root_mean_squared_error: 0.0166 - val_loss: 2.4008e-04 - val_root_mean_squared_error: 0.0155
Epoch 51/100
256/256 [==============================] - 15s 57ms/step - loss: 2.

KeyboardInterrupt: 

In [28]:
#Saving model
#Saving model
model.save("Depth_8x8x8_autoencoder.h5")

In [30]:
size_f = 1024
size_r = 64
#Assuming square
splits = int(size_f/size_r)

full_num_img = len(dataloader.classfiles)
im_indices = [a for a in range(full_num_img)]


from sklearn.model_selection import train_test_split

samp_per_img = 64

split_range = [a for a in range(splits*splits)]
samp = train_test_split(split_range,train_size = samp_per_img)[0]

X = np.zeros((full_num_img*samp_per_img,size_r,size_r,1))
y = np.zeros((full_num_img*samp_per_img,size_r,size_r),bool)
for a in im_indices:
    c_ind = a*samp_per_img
    tmp = dataloader.get_data("depth",[a])[0]
    tmp = np.moveaxis([tmp],(0,1,2),(2,0,1))
    c_img = split_image(tmp,splits)
    c_class_tmp = dataloader.get_data("class",[a])[0]
    c_class = split_image(cv2.resize(c_class_tmp,(size_f,size_f)),splits)[:,:,:,0].astype(bool)
    X[c_ind:c_ind+samp_per_img,:,:,:] = c_img[samp]
    y[c_ind:c_ind+samp_per_img,:,:] = c_class[samp]
y = np.moveaxis(np.asarray([y],bool),(0,1,2,3),(3,0,1,2))

X = np.asarray([(a-np.min(a,0))/(np.max(a)-np.min(a)) for a in X])


#Splitting into train and test data
test_size = 0.1
val_size = 0.1

shape_range = [a for a in range(X.shape[0])]

t_ind,test_ind = train_test_split(shape_range,test_size=test_size)
train_ind,val_ind = train_test_split(t_ind,test_size=val_size)

X_train,y_train = X[train_ind],y[train_ind]
X_val,y_val = X[val_ind],y[val_ind]
X_test,y_test = X[test_ind],y[test_ind]

In [31]:
from tensorflow import keras

model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters=32,kernel_size=7,input_shape = [size_r,size_r,1],
                              activation="tanh", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.MaxPool2D(2, padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=8,kernel_size=3,activation="relu", padding="same"))

#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
#model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(filters=64,kernel_size=3,activation="relu", padding="same"))
model.add(keras.layers.Conv2DTranspose(32, 3, strides=2, activation="relu", padding="same"))
model.add(keras.layers.Conv2D(1,kernel_size=3,activation="sigmoid", padding="same"))

model.compile(loss="mean_squared_error",metrics = [keras.metrics.RootMeanSquaredError()],
             optimizer="nadam")

In [ ]:
history = model.fit(X_train,X_train,epochs=100,validation_data=(X_val,X_val),batch_size=32)

Epoch 1/100
128/128 [==============================] - 28s 213ms/step - loss: 0.0244 - root_mean_squared_error: 0.1561 - val_loss: 0.0038 - val_root_mean_squared_error: 0.0616
Epoch 2/100
128/128 [==============================] - 28s 217ms/step - loss: 0.0039 - root_mean_squared_error: 0.0626 - val_loss: 0.0022 - val_root_mean_squared_error: 0.0465
Epoch 3/100
128/128 [==============================] - 27s 214ms/step - loss: 0.0027 - root_mean_squared_error: 0.0516 - val_loss: 0.0022 - val_root_mean_squared_error: 0.0464
Epoch 4/100
128/128 [==============================] - 27s 214ms/step - loss: 0.0020 - root_mean_squared_error: 0.0446 - val_loss: 0.0011 - val_root_mean_squared_error: 0.0339
Epoch 5/100
128/128 [==============================] - 27s 214ms/step - loss: 0.0019 - root_mean_squared_error: 0.0432 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0383
Epoch 6/100
128/128 [==============================] - 27s 213ms/step - loss: 0.0014 - root_mean_squared_error: 0.0372 -

In [ ]:
model.save("Depth_64x64x8_autoencoder.h5")